In [4]:
import pickle
import pandas as pd
import numpy as np

In [2]:
with open('df1.pkl', 'rb') as f:
    data = pickle.load(f)
df = pd.DataFrame(data)
df

,bowler,batsman,wicket,over,outcome
0,Shafiul,Harris,0,1,2
1,Shafiul,Hussey,1,1,7
2,Shafiul,Hussey,1,1,2
3,Shafiul,Smith,1,1,1
4,Shafiul,Hussey,1,1,1
...,...,...,...,...,...
56730,Tanvir,Sibanda,6,20,1
56731,Tanvir,Sibanda,6,20,4
56732,Tanvir,Sibanda,6,20,0
56733,Tanvir,Sibanda,6,20,2


In [5]:
X = np.load("X_mat.npy")

In [6]:
balls_faced = X.sum(axis=3)

In [7]:
balls_faced.shape

(436, 22, 11)

: 